In [9]:
from openai import OpenAI
import json
import re
from datasets import load_dataset
from dotenv import load_dotenv
import os

In [10]:
load_dotenv()

key = os.getenv("API_KEY")
client = OpenAI(api_key=key, base_url="https://api.deepseek.com")

In [11]:
dataset = load_dataset("eriktks/conll2003")
#print(dataset['train']['id'])
#print(dataset['train']['tokens'])
#print(dataset['train']['pos_tags'])
#print(dataset['train']['chunk_tags'])
#print(dataset['train']['ner_tags'][:15])

In [12]:
ner_dict = {'O': 0, 'B-PER': 1, 'I-PER': 2, 'B-ORG': 3, 'I-ORG': 4, 'B-LOC': 5, 'I-LOC': 6, 'B-MISC': 7, 'I-MISC': 8}

In [13]:
def get_ner_label(value, ner_dict):
    for k, v in ner_dict.items():
        if v == value:
            return k
    return None

In [14]:
print(dataset['train']['tokens'][0])
print([get_ner_label(x, ner_dict) for x in dataset['train']['ner_tags'][0]])

['EU', 'rejects', 'German', 'call', 'to', 'boycott', 'British', 'lamb', '.']
['B-ORG', 'O', 'B-MISC', 'O', 'O', 'O', 'B-MISC', 'O', 'O']


In [ ]:
def send_prompt(prompt):
    response = client.chat.completions.create(
        model="deepseek-chat",
        messages=[
            #{"role": "system", "content": "You are a helpful assistant"},
            {"role": "user", "content": prompt},
        ],
        stream=False
    )

    return response.choices[0].message.content

### VANILLA METHOD

In [ ]:
for sentence in dataset['train']['tokens']:
    prompt = f"Given the entity label set: {list(ner_dict.keys())}.\n \
        Based on the given entity label set, please recognize the named entities in the given text.\n \
        Return only a list of tuples with each token and its label, nothing else. For example [('In','O'), ('America','I-LOC'), ('is','O'), ('cold','O'), ...]\n \
        Text: {" ".join(sentence)}"

    answer = send_prompt(prompt)
    # print(answer)
    # tags = re.findall(r"\('.*?',\s*'(.*?)'\)", answer)
    # tags = [ner_dict[tag] for tag in tags]
    # print(tags)
    with open("vanilla.txt", "a") as file:
        file.write(f"{answer}\n")


[('EU', 'B-ORG'), ('rejects', 'O'), ('German', 'B-MISC'), ('call', 'O'), ('to', 'O'), ('boycott', 'O'), ('British', 'B-MISC'), ('lamb', 'O'), ('.', 'O')]
[3, 0, 7, 0, 0, 0, 7, 0, 0]
[('Peter', 'B-PER'), ('Blackburn', 'I-PER')]
[1, 2]


### Q&A